This and other notebooks are scratch pads to try ideas out or debug small features before integrating with more proper scripts

### Some custom functions

#### Archs - prototypes

More archs are imported from archs.py

**Wish list**: equivalent to $NT + C_{opp}$

How this will work could be tetchy: must be at least loosely interpretable as having to do with game theoretic value, but value is not reflected in network output. 

However, we can separate softmax nonlinearity and apply rectified linear instead to get a "value" representation for each position (!). Then, we can repeat with same network by (1) adding argmax to input, (2) swapping input channels, and (3) passing back through the network. We can then do one of several things (optionally?) scale the negative opponent response, add, and softmax

Another option is to change (pseudo-)output layer to be a 72 vector to hold opponent values for true min-maxing. This might actually be better, but is more difficult to compare to old architecture. 

**Wish list**: non random init to HS-like features

In [ ]:
def thinky_agent(nfil=32, input_var=None):
    """Theano graph for convnet that adds opponent; WIP"""
    
    class FixLayer(lasagne.layers.Layer):
        def get_output_for(self, input, **kwargs):
            corrector = (1 - input_var.sum(axis=1)).reshape((input_var.shape[0], 36))
            numer = input * corrector
            return numer / numer.sum(axis=1).dimshuffle((0, 'x'))
        
    class ValueLayer(lasagne.layers.ElemwiseSumLayer):
        def get_output_for(self, inputs, **kwargs):
            pass
        
        def get_output_shape_for(self, input_shapes):
            pass
        
    input_shape=(None, 2, 4, 9)
    input_layer = lasagne.layers.InputLayer(shape=input_shape, input_var=input_var)
    
    network = lasagne.layers.Conv2DLayer(
        input_layer, 
        num_filters=nfil, filter_size=(4,4), pad='full',
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform()
    )
    
    network = lasagne.layers.DropoutLayer(network, p=.75)
    
    network = lasagne.layers.NonlinearityLayer(
        network, 
        nonlinearity=lasagne.nonlinearities.softmax
    )
    
    network = FixLayer(network)
    
    return network

In [ ]:
# NOTE: it is not possible with lasagne to concat with different filter shapes :(
#     pool_layer_1_1 = L.layers.FeaturePoolLayer(conv_layer_1_1, pool_size=2)
    
#     conv_layer_1_2 = L.layers.Conv2DLayer(
#         input_layer, num_filters=32, filter_size=(4,8), pad='full', 
#         nonlinearity=L.nonlinearities.leaky_rectify,
#         W=lasagne.init.HeUniform(gain='relu'),
#     )
    
#     pool_layer_1_2 = L.layers.FeaturePoolLayer(conv_layer_1_2, pool_size=2, axis=-1)
    
    
#     layer_1 = L.layers.ConcatLayer([pool_layer_1_1, pool_layer_1_2])
    
#     pool_layer_1 = L.layers.MaxPool2DLayer(conv_layer_1, pool_size=(2, 2), stride=-1)

In [ ]:
import sys
import os
import time
import numpy as np
import theano
import theano.tensor as T
import lasagne as L
import pandas as pd
from util import *
from archs import *
from load_data import *
import train as tr
from network import MNKNet

import matplotlib.pyplot as plt
import seaborn as sns

headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
datafile = os.path.join(headdir, 'Data/0_hvh/Clean/_summaries/model_input_with_groups.csv')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nns')

sns.set_style('white')
sns.set_context('poster')
%matplotlib inline

### Prototype box

In [ ]:
def prototype(nfil=None, input_var=None, return_layers=False):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)
    
    layers = []
    
    input_layer = L.layers.InputLayer(shape=input_shape, input_var=input_var)
    
    conv_layer_1 = L.layers.Conv2DLayer(
        input_layer, num_filters=128, filter_size=(2, 2), pad='full', 
        nonlinearity=L.nonlinearities.leaky_rectify,
        W=L.init.HeUniform(gain='relu'),
    )
    
    conv_layer_1 = L.layers.DropoutLayer(conv_layer_1, p=.75)
    
    pool_layer_1 = L.layers.FeaturePoolLayer(conv_layer_1, pool_size=2)
    
    
    dense_layer_1 = L.layers.DenseLayer(
        pool_layer_2, num_units=128, nonlinearity=L.nonlinearities.leaky_rectify,
        W = L.init.HeUniform(gain='relu')
    )
    
    dense_layer_2 = L.layers.DenseLayer(
        dense_layer_1, num_units=36, nonlinearity=L.nonlinearities.leaky_rectify,
        W = L.init.HeUniform(gain='relu')
    )
    
    softmax_output = L.layers.NonlinearityLayer(dense_layer_2, nonlinearity=L.nonlinearities.softmax)
    network = FixLayer(softmax_output)
    
    return network

In [ ]:
def prototype(nfil=None, input_var=None, return_layers=False):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)
        
    input_layer = L.layers.InputLayer(shape=input_shape, input_var=input_var)
    
    network = L.layers.Conv2DLayer(
        input_layer, num_filters=32, filter_size=(4, 4), pad='full', 
        nonlinearity=L.nonlinearities.leaky_rectify,
        W=L.init.HeUniform(gain='relu'),
    )
        
#     network = L.layers.FeaturePoolLayer(network, pool_function=T.sum, pool_size=2)
    
    network = L.layers.DropoutLayer(network, p=.75)
    
#     network = L.layers.DenseLayer(
#         network, num_units=128, nonlinearity=L.nonlinearities.leaky_rectify,
#         W=L.init.HeUniform(gain='relu')
#     )
    
#     network = L.layers.DropoutLayer(network, p=.7)
    
    network = L.layers.DenseLayer(
        network, num_units=36, nonlinearity=L.nonlinearities.leaky_rectify,
        W = L.init.HeUniform(gain='relu')
    )
    
    softmax_output = L.layers.NonlinearityLayer(network, nonlinearity=L.nonlinearities.softmax)
    network = FixLayer(softmax_output)
    
    return network

### Single architecture training

For prototyping

In [ ]:
np.random.seed(985227)

ARCH = prototype
LVL = 32 #None for softmax
arch = lambda input_var=None: ARCH(nfil=LVL, input_var=input_var)

CV_nlls, traces, nets = tr.CV_train(
    arch, 
    batchsize=128, 
    epochs=5000, 
    thresh=100,
    everyn=50,
    custom_loaded=CV_loader(datafile, norm_input=False)
)

## Save data

In [ ]:
n = nets[3]

In [ ]:
n.save_params('sample_network')

In [ ]:
qvar = T.dmatrix('dist')
pvar = T.ivector('targ')
loss = lasagne.objectives.categorical_crossentropy(qvar, pvar)
loss_fn = theano.function([qvar, pvar], loss)

In [ ]:
nets[0].data = nets[0].data.drop(d.columns[~d.columns.isin(['subject', 'color', 'bp', 'wp', 'response', 'rt', 'splitg'])], axis=1)
d = nets[0].data
d.loc[d['color']==0, 'own'] = d.loc[d['color']==0, 'bp']
d.loc[d['color']==0, 'opp'] = d.loc[d['color']==0, 'wp']
d.loc[d['color']==1, 'own'] = d.loc[d['color']==1, 'wp']
d.loc[d['color']==1, 'opp'] = d.loc[d['color']==1, 'bp']
for j in range(36):
    d[str(j)] = np.nan

for i in np.arange(1, 6, 1):
    c = d['splitg']==i
    dd = d.loc[c]
    dp = dd['own'] + dd['opp']
    dp = dp.map(lambda x: np.array(list(x)).astype(int).reshape([1, 2, 4, 9]))
    dp = np.concatenate(dp.values)
    net_output = nets[i-1].output_fn(dp)
    d.loc[c, [str(j) for j in range(36)]] = net_output   
    d.loc[c, 'nll'] = loss_fn(net_output, d.loc[c, 'response'])
    
d[['subject', 'splitg', 'nll', 'response'] + [str(i) for i in range(36)]].to_csv(os.path.join(resultsdir, 'smart_32.csv'))

In [ ]:
from scipy.stats import bayes_mvs
bayes_mvs(d['nll'])

### Architecture comparison

**Don't run this section** unless you want to train 18 different networks. Load params or remove loop to train the architecture you want instead.

In [ ]:
np.random.seed(985227)
datafile = '../../Google Drive/Bas Zahy Gianni - Games/Data/0_hvh/Clean/_summaries/model_input_with_groups.csv'

CVN, TRACES, NETS = ([],)*3
ARCHS = [naive_agent, smart_agent]
ARCH_NAMES = ['naive', 'smart']
LVLS = 2**np.arange(9)


for lvl in LVLS:
    for aidx, arch in enumerate(ARCHS):
        np.random.seed(985227)
        CV_nlls, traces, nets = tr.CV_train(
            lambda input_var=None: arch(nfil=lvl, input_var=input_var),
            batchsize=500, 
            epochs=5000, 
            thresh=100,
            everyn=5000,
            custom_loaded=custom_loader(datafile)
        )
        
        CVN.append(CV_nlls)
        TRACES.append(traces)
        NETS.append(nets)
        
        print(
            '\n',
            ARCH_NAMES[aidx] + ' ' + 'LVL' + ' ' + str(lvl) + ':\t',
            np.mean(CV_nlls),
            '\n'
        )
        
        save_appended_data(
            nets, 
            ARCH_NAMES[aidx] + '_' + str(lvl)
        )

### Save traces and parameters

In [ ]:
CVN = CVN[::3]
TRACES = TRACES[1::3]
NETS = NETS[2::3]

In [ ]:
for i, t in enumerate(TRACES):
    np.savetxt(
        './results/trace_' + str(i) + '.csv', 
        np.array(t).reshape([15, 5000]).T, 
        delimiter=','
    )
    
for i, n in enumerate(NETS):
    for j, nn in enumerate(n):
        nn.save_params('./results/params_' + str(i*len(nets) + j))

# Some unorganized junk

### Separate data file

In [ ]:
datadir = '../../Google Drive/Bas Zahy Gianni - Games/Data/0_hvh/Clean/_summaries/'
datafile = datadir + 'model_input_with_groups.csv'
datatypes = [('subject', 'i4'), ('color', 'i4'), 
                 ('bp', 'S36'), ('wp', 'S36'), 
                 ('response', 'i4'), ('rt', 'i4'), ('splitg', 'i4')]
data = np.loadtxt(datafile, delimiter=',', dtype=datatypes)
data = pd.DataFrame.from_records(data)
decoder = lambda x: x.decode('utf-8')
data.loc[:, 'bp'] = data.loc[:, 'bp'].map(decoder)
data.loc[:, 'wp'] = data.loc[:, 'wp'].map(decoder)
# data.loc[data.subject==0, :].to_csv(datadir + 'sub0.csv', header=False, index=False)

### Load Params (if you skipped training)

(to do! pretty easy though)

### Save transferable params

(needs updated)

In [ ]:
params = lasagne.layers.get_all_param_values(n.network)

for i, p in enumerate(params):
    ps = p.shape
    pp = p.reshape([ps[0], np.product(ps[1:])])
    np.savetxt(str(i//2) + str(i%2)+'b', pp, fmt='%.10f')

# Figures

In [ ]:
np.savetxt('./results/smart_32_traces_training.txt', np.array(traces)[:, 0, :1000].reshape((5, 1000)).T)
np.savetxt('./results/smart_32_traces_validate.txt', np.array(traces)[:, 1, :1000].reshape((5, 1000)).T)

In [ ]:
np.array(traces).shape

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8.5, 5.5))
axes.plot(traces[0][0][:500])
axes.plot(traces[1][1][:500])
sns.despine()

TODO: move to the results notebook

In [ ]:
def agg_nlls(data):
    meanagg = pd.pivot_table(data=data, 
                   values='cnn_nll', index='n_pieces', 
                   aggfunc=np.mean)
    sdagg = pd.pivot_table(data=data,
                          values='cnn_nll', index='n_pieces',
                          aggfunc=np.std)
    return meanagg, sdagg

figs, axes = plt.subplots(10, 4, figsize=(18,24), 
                          sharex=True, sharey=True)

for p in data.subject.unique():
    ax = axes.flatten()[p]
    m, sd = agg_nlls(data.loc[data.subject==p])
    ax.errorbar(m.index, m.values, yerr=sd.values)
    ax.set_ylim([0,8])
    
sns.despine()

In [ ]:
m, sd = agg_nlls(data)
plt.errorbar(m.index, m.values, yerr=sd.values, ecolor=(.8, .9, .8))
ax = plt.gca()
ax.set_ylim([0,4.2])
ax.set_xlim([-.5, 35.5])
sns.despine()
fig = plt.gcf()
fig.savefig('./nll_against_n_pieces')

In [ ]:
pos = 506
print(data.loc[pos, 'subject'])
fig, axes = plt.subplots(1, 1, figsize=(20, 10))
d = data.loc[pos, positions].values.reshape([4,9]).astype(float)
print(d)
sns.heatmap(
    d, square=True, vmin=0, vmax=.25, cbar=False,
    xticklabels=False, yticklabels=False, ax=axes, annot=True
)

reconstitute = lambda x: np.array(list(map(int, x))).reshape(4,9)

if data.loc[pos, 'color'] == 0:
    own_color, opp_color = 'black', 'white'
else:
    own_color, opp_color = 'white', 'black'

p = np.where(reconstitute(data.loc[pos, 'bp'])==1)
plt.scatter( .5 + p[1], 3.5 - p[0], c=own_color, s=10000)
p = np.where(reconstitute(data.loc[pos, 'wp'])==1)
plt.scatter( .5 + p[1], 3.5 - p[0], c=opp_color, s=10000)
r = data.loc[pos, 'response']
p = (r % 9, r // 9)
plt.scatter(p[0] + .5, 3.5 - p[1], c=(.7, .7, .7), s=10000);
fig.savefig('nice example.png', bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10,6), squeeze=False)
ep = net.last_epoch
ax = axes[0,0]
ax.plot(np.arange(net.tr_nll.size)[:ep], net.tr_nll[:ep]/33, 
         label='Training set NLL', color='grey')
ax.plot(np.arange(net.val_nll.size)[:ep], net.val_nll[:ep]/5, 
         label='Validation set NLL', color='black')
ax.plot([0, 200], [3.58, 3.58], 'k--', label='Random guessing')
lgd = ax.legend(bbox_to_anchor=(1.45, .65))
ax.set_xlabel('Training Epoch')
ax.set_ylabel('NLL')
ax.set_title('Overfitting on combined data')
sns.despine()
fig.savefig('training_slope.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(64, 2, figsize=(10, 192), squeeze=False)

filters = lasagne.layers.get_all_param_values(n.network)[0]

for idx in range(64):
    sns.heatmap(filters[idx,0], square=True, cmap='Greys', fmt= '.2f', 
                vmin=-1.5, vmax=1.5, annot=True,
                ax=axes[idx,0], cbar=False, xticklabels=False, yticklabels=False)
    sns.heatmap(filters[idx,1], square=True, cmap='Greys', fmt='.2f', 
                vmin=-1.5, vmax=1.5, annot=True,
                ax=axes[idx,1], cbar=False, xticklabels=False, yticklabels=False)

fig.savefig('filters.png', bbox_inches='tight')